<a href="https://colab.research.google.com/github/lnrdmnc/NER-NLP/blob/main/spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduzione

Questo tutorial fornisce una breve introduzione al lavoro con il linguaggio naturale (talvolta chiamato "text analytics") in Python, utilizzando [spaCy](https://spacy.io/) e le librerie correlate.
I team che si occupano di scienza dei dati nell'industria devono lavorare con molti testi, una delle quattro principali categorie di dati utilizzati nell'apprendimento automatico.
Di solito si tratta di testo generato dall'uomo, anche se non sempre.

Pensateci: come funziona il "sistema operativo" delle aziende? In genere, ci sono contratti (di vendita, di lavoro, di collaborazione), fatture, polizze assicurative, regolamenti e altre leggi, e così via.
Tutti questi elementi sono rappresentati sotto forma di testo.

Potreste imbattervi in alcuni acronimi: _elaborazione del linguaggio naturale_ (NLP), _comprensione del linguaggio naturale_ (NLU), _generazione del linguaggio naturale_ (NLG) - che sono rispettivamente "leggere il testo", "comprendere il significato", "scrivere il testo".
Sempre più spesso questi compiti si sovrappongono e diventa difficile categorizzare qualsiasi caratteristica.

Il framework _spaCy_, insieme a un'ampia e crescente gamma di plug-in e altre integrazioni, fornisce funzionalità per un'ampia gamma di compiti di linguaggio naturale.
È diventata una delle librerie di linguaggio naturale più utilizzate in Python per i casi d'uso dell'industria e ha una comunità piuttosto numerosa e, con essa, un grande sostegno per la commercializzazione dei progressi della ricerca, dato che quest'area continua a evolversi rapidamente.

## Getting Started
[note di installazione](https://spacy.io/usage) per un "configuratore" che genera i comandi di installazione in base alle piattaforme e ai linguaggi naturali da supportare.

Alcuni tendono a usare `pip` mentre altri usano `conda`, e ci sono istruzioni per entrambi.  Per esempio, per iniziare a usare _spaCy_ che lavora con testi in inglese ed è installato tramite `conda` su un sistema Linux:
```
conda install -c conda-forge spacy
python -m spacy download en_core_web_sm
```

BTW, la seconda riga qui sopra è un download di risorse linguistiche (modelli, ecc.) e il `_sm` alla fine del nome del download indica un modello "piccolo". Ci sono anche "medium" e "large", anche se questi sono piuttosto grandi. Alcune delle funzioni più avanzate dipendono da quest'ultimo, anche se in questo (breve) tutorial non ci immergeremo fino in fondo a questo oceano.

Ora carichiamo _spaCy_ ed eseguiamo un po' di codice:

In [2]:
import spacy

nlp = spacy.load("en_core_web_sm")

La variabile nlp è ora la porta d'accesso a tutto ciò che è spaCy ed è caricata con il piccolo modello en_core_web_sm per l'inglese. Quindi, facciamo passare un piccolo "documento" attraverso il parser del linguaggio naturale:

In [3]:
text = "The rain in Spain falls mainly on the plain."
doc = nlp(text)

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.is_stop)

The the DET True
rain rain NOUN False
in in ADP True
Spain Spain PROPN False
falls fall VERB False
mainly mainly ADV False
on on ADP True
the the DET True
plain plain NOUN False
. . PUNCT False


Per prima cosa abbiamo creato un [doc](https://spacy.io/api/doc) dal testo, che è un contenitore per un documento e tutte le sue annotazioni. Poi abbiamo iterato il documento per vedere cosa _spaCy_ aveva analizzato.

Bene, ma sono molte informazioni e un po' difficili da leggere. Riformuliamo l'analisi di _spaCy_ di quella frase come un dataframe [pandas](https://pandas.pydata.org/):

In [4]:
import pandas as pd

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []

for t in doc:
    row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)

Molto più leggibile!
In questo semplice caso, l'intero documento è solo una breve frase.
Per ogni parola della frase, _spaCy_ ha creato un [token](https://spacy.io/api/token), e in ogni token sono stati inseriti dei campi da mostrare:

 - testo grezzo
 - [lemma](https://en.wikipedia.org/wiki/Lemma_(morfologia)) - una forma radicale della parola
 - [parte del discorso](https://en.wikipedia.org/wiki/Part_of_speech)
 - un flag che indica se la parola è una _stopword_, cioè una parola comune che può essere filtrata

Utilizziamo poi la libreria [displaCy](https://ines.io/blog/developing-displacy) per visualizzare l'albero di analisi della frase:

In [5]:
from spacy import displacy

displacy.render(doc, style="dep", jupyter=True)

Quando _spaCy_ crea un documento, utilizza un principio di _tokenizzazione non distruttiva_ che significa che i token, le frasi, ecc. sono semplicemente indici in un lungo array. In altre parole, non taglia il flusso di testo in piccoli pezzi. Quindi ogni frase è uno [span](https://spacy.io/api/span) con un indice di inizio e uno di fine nell'array del documento:

In [6]:
for sent in doc.sents:
    print(">", sent.start, sent.end)

> 0 10


Possiamo indicizzare l'array del documento per estrarre i token di una frase:

In [7]:
doc[48:54]

In [8]:
token = doc[51]
print(token.text, token.lemma_, token.pos_)

IndexError: [E026] Error accessing token at position 51: out of bounds in Doc of length 10.

A questo punto possiamo analizzare un documento, segmentarlo in frasi, quindi esaminare le annotazioni sui token di ciascuna frase. È un buon inizio.

## Acquisizione del testo

Ora che possiamo analizzare i testi, dove li troviamo?
Una fonte rapida è quella di sfruttare il web.
Naturalmente, quando scarichiamo le pagine web, otteniamo l'HTML e dobbiamo estrarre il testo da esse.
[Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) è un pacchetto popolare per questo scopo.

Prima di tutto, un po' di pulizia:

In [9]:
import sys
import warnings

warnings.filterwarnings("ignore")

Di seguito sono riportati esempi di utilizzo di [codec](https://docs.python.org/3/library/codecs.html) e [normalize unicode](https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize). NB: il testo di esempio proviene dall'articolo "[Metal umlat](https://en.wikipedia.org/wiki/Metal_umlaut)".

In [10]:
x = "Rinôçérôse screams ﬂow not unlike an encyclopædia, \
'TECHNICIÄNS ÖF SPÅCE SHIP EÅRTH THIS IS YÖÜR CÄPTÅIN SPEÄKING YÖÜR ØÅPTÅIN IS DEA̋D' to Spın̈al Tap."

type(x)

str

La variabile x è una stringa in Python:

In [11]:
repr(x)

'"Rinôçérôse screams ﬂow not unlike an encyclopædia, \'TECHNICIÄNS ÖF SPÅCE SHIP EÅRTH THIS IS YÖÜR CÄPTÅIN SPEÄKING YÖÜR ØÅPTÅIN IS DEA̋D\' to Spın̈al Tap."'

La sua traduzione in [ASCII](http://www.asciitable.com/) è inutilizzabile dai parser:

In [12]:
ascii(x)

'"Rin\\xf4\\xe7\\xe9r\\xf4se screams \\ufb02ow not unlike an encyclop\\xe6dia, \'TECHNICI\\xc4NS \\xd6F SP\\xc5CE SHIP E\\xc5RTH THIS IS Y\\xd6\\xdcR C\\xc4PT\\xc5IN SPE\\xc4KING Y\\xd6\\xdcR \\xd8\\xc5PT\\xc5IN IS DEA\\u030bD\' to Sp\\u0131n\\u0308al Tap."'

La codifica in [UTF-8](http://unicode.org/faq/utf_bom.html) non aiuta molto:
Ignorare i caratteri difficili è forse una strategia ancora peggiore:

In [13]:
x.encode('utf8')

b"Rin\xc3\xb4\xc3\xa7\xc3\xa9r\xc3\xb4se screams \xef\xac\x82ow not unlike an encyclop\xc3\xa6dia, 'TECHNICI\xc3\x84NS \xc3\x96F SP\xc3\x85CE SHIP E\xc3\x85RTH THIS IS Y\xc3\x96\xc3\x9cR C\xc3\x84PT\xc3\x85IN SPE\xc3\x84KING Y\xc3\x96\xc3\x9cR \xc3\x98\xc3\x85PT\xc3\x85IN IS DEA\xcc\x8bD' to Sp\xc4\xb1n\xcc\x88al Tap."

Ignorare i caratteri difficili è forse una strategia ancora peggiore:

In [14]:
x.encode('ascii', 'ignore')

b"Rinrse screams ow not unlike an encyclopdia, 'TECHNICINS F SPCE SHIP ERTH THIS IS YR CPTIN SPEKING YR PTIN IS DEAD' to Spnal Tap."

Tuttavia, si può *normalizzare* il testo, quindi codificare...

In [15]:
import unicodedata

unicodedata.normalize('NFKD', x).encode('ascii','ignore')


b"Rinocerose screams flow not unlike an encyclopdia, 'TECHNICIANS OF SPACE SHIP EARTH THIS IS YOUR CAPTAIN SPEAKING YOUR APTAIN IS DEAD' to Spnal Tap."

Anche prima di questa normalizzazione e codifica, potrebbe essere necessario convertire esplicitamente alcuni caratteri **prima** del parsing. Per esempio:

In [16]:
x = "The sky “above” the port … was the color of ‘cable television’ – tuned to the Weather Channel®"
ascii(x)

"'The sky \\u201cabove\\u201d the port \\u2026 was the color of \\u2018cable television\\u2019 \\u2013 tuned to the Weather Channel\\xae'"

Considerate i risultati di questa riga:

In [17]:
unicodedata.normalize('NFKD', x).encode('ascii', 'ignore')

b'The sky above the port ... was the color of cable television  tuned to the Weather Channel'

...che ancora tralascia caratteri che potrebbero essere importanti per l'analisi di una frase.

Quindi un approccio più avanzato potrebbe essere:

In [18]:
x = x.replace('“', '"').replace('”', '"')
x = x.replace("‘", "'").replace("’", "'")
x = x.replace('…', '...').replace('–', '-')
x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8')
print(x)

The sky "above" the port ... was the color of 'cable television' - tuned to the Weather Channel


Parsing HTML

In [19]:
from bs4 import BeautifulSoup
import requests
import traceback

def get_text (url):
    buf = []

    try:
        soup = BeautifulSoup(requests.get(url).text, "html.parser")

        for p in soup.find_all("p"):
            buf.append(p.get_text())

        return "\n".join(buf)
    except:
        print(traceback.format_exc())
        sys.exit(-1)

Ora prendiamo alcuni testi da fonti online.
Possiamo confrontare le licenze open source ospitate sul sito della [Open Source Initiative](https://opensource.org/licenses/):

In [20]:
lic = {}
lic["mit"] = nlp(get_text("https://opensource.org/licenses/MIT"))
lic["asl"] = nlp(get_text("https://opensource.org/licenses/Apache-2.0"))
lic["bsd"] = nlp(get_text("https://opensource.org/licenses/BSD-3-Clause"))

for sent in lic["bsd"].sents:
    print(">", sent)

> Search


								SPDX short identifier:
								BSD-3-Clause							

Note: This license has also been called the “New BSD License” or “Modified BSD License”.
> See also the 2-clause BSD License.

> Copyright <YEAR> <COPYRIGHT HOLDER>
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
1.
> Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

> 2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

> 3.
> Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

> THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS “AS IS” AND
> A

Un caso d'uso comune per il lavoro con il linguaggio naturale è quello di confrontare i testi. Ad esempio, con le licenze open source possiamo scaricare i testi, analizzarli e poi confrontare le metriche di [similarità](https://spacy.io/api/doc#similarity) tra loro:

In [21]:
pairs = [
    ["mit", "asl"],
    ["asl", "bsd"],
    ["bsd", "mit"]
]

for a, b in pairs:
    print(a, b, lic[a].similarity(lic[b]))

mit asl 0.8898028126415602
asl bsd 0.8789439210898888
bsd mit 0.970449166310577


Questo è interessante, poiché le licenze [BSD](https://opensource.org/licenses/BSD-3-Clause) e [MIT](https://opensource.org/licenses/MIT) sembrano essere i documenti più simili.
In effetti sono strettamente correlate.

È vero che in ogni documento è stato incluso del testo aggiuntivo a causa della clausola di esclusione di responsabilità OSI nel piè di pagina, ma questo fornisce un'approssimazione ragionevole per confrontare le licenze.

## Natural Language Understanding
Ora analizziamo alcune delle caratteristiche più interessanti per la NLU.
Dato che abbiamo il parsing di un documento, da un punto di vista puramente grammaticale possiamo estrarre i [noun chunks](https://spacy.io/usage/linguistic-features#noun-chunks), cioè ciascuna delle frasi di sostantivo:

In [22]:
text = "Steve Jobs and Steve Wozniak incorporated Apple Computer on January 3, 1977, in Cupertino, California."
doc = nlp(text)

for chunk in doc.noun_chunks:
    print(chunk.text)

Steve Jobs
Steve Wozniak
Apple Computer
January
Cupertino
California


Non male. Le frasi dei sostantivi in una frase forniscono generalmente un maggior contenuto informativo, come un semplice filtro utilizzato per ridurre un lungo documento in una rappresentazione più "distillata".

Possiamo portare avanti questo approccio e identificare le [named entities](https://spacy.io/usage/linguistic-features#named-entities) all'interno del testo, cioè i nomi propri:

In [23]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Steve Jobs PERSON
Steve Wozniak PERSON
Apple Computer ORG
January 3, 1977 DATE
Cupertino GPE
California GPE


La libreria _displaCy_ offre un modo eccellente per visualizzare le entità denominate:

In [24]:
displacy.render(doc, style="ent", jupyter=True)

Se si lavora con applicazioni [knowledge graph](https://www.akbc.ws/) e altri [linked data](http://linkeddata.org/), la sfida consiste nel costruire collegamenti tra le entità nominate in un documento e altre informazioni correlate alle entità, il che si chiama [entity linking](http://nlpprogress.com/english/entity_linking.html).
L'identificazione delle entità nominate in un documento è il primo passo di questo particolare tipo di lavoro di intelligenza artificiale.
Per esempio, dato il testo qui sopra, si potrebbe collegare l'entità denominata `Steve Wozniak' a un [lookup in DBpedia](http://dbpedia.org/page/Steve_Wozniak).

In termini più generali, si possono anche collegare i lemmi alle risorse che ne descrivono il significato.
Per esempio, in una prima sezione abbiamo analizzato la frase "I gorilla si sono scatenati" e siamo riusciti a dimostrare che il lemma della parola "si sono scatenati" è il verbo "andare". A questo punto possiamo utilizzare un venerabile progetto chiamato [WordNet](https://wordnet.princeton.edu/) che fornisce un database lessicale per l'inglese - in altre parole, è un thesaurus computabile.

Esiste un'integrazione _spaCy_ per WordNet chiamata
[spacy-wordnet](https://github.com/recognai/spacy-wordnet) di [Daniel Vila Suero](https://twitter.com/dvilasuero), un esperto di linguaggio naturale e di grafi di conoscenza.

Poi caricheremo i dati di WordNet tramite NLTK (sono cose che succedono):

In [25]:
import nltk

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

https://colab.research.google.com/drive/10cEYZS4I_nua081chK6MHeocGpdLBmSf#scrollTo=Cn3jthvsqn-b&line=1&uniqifier=1

In [26]:
!pip install spacy-wordnet

In [30]:
from spacy_wordnet.wordnet_annotator import WordnetAnnotator

print("before", nlp.pipe_names)

if "WordnetAnnotator" not in nlp.pipe_names:
    nlp.add_pipe(WordnetAnnotator(nlp.lang), after="tagger")

print("after", nlp.pipe_names)

before ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


TypeError: WordnetAnnotator.__init__() missing 1 required positional argument: 'name'

Nella lingua inglese, alcune parole sono famose per avere molti significati possibili. Ad esempio, si possono consultare i risultati di una ricerca online su [WordNet](http://wordnetweb.princeton.edu/perl/webwn?s=star&sub=Search+WordNet&o2=&o0=1&o8=1&o1=1&o7=&o5=&o9=&o6=&o3=&o4=&h=) per trovare i significati relativi alla parola `ritirare`.

Ora usiamo _spaCy_ per eseguire questa ricerca automaticamente:

In [31]:
token = nlp("withdraw")[0]
token._.wordnet.synsets()

AttributeError: [E046] Can't retrieve unregistered extension attribute 'wordnet'. Did you forget to call the `set_extension` method?

In [32]:
token._.wordnet.lemmas()

AttributeError: [E046] Can't retrieve unregistered extension attribute 'wordnet'. Did you forget to call the `set_extension` method?

In [33]:
token._.wordnet.wordnet_domains()

AttributeError: [E046] Can't retrieve unregistered extension attribute 'wordnet'. Did you forget to call the `set_extension` method?

Ancora una volta, se si lavora con i grafi di conoscenza, i collegamenti al "senso delle parole" di WordNet possono essere utilizzati insieme agli algoritmi dei grafi per aiutare a identificare i significati di una particolare parola. Questo può anche essere usato per sviluppare riassunti per sezioni più ampie di testo, attraverso una tecnica chiamata riassunto. Si tratta di una tecnica che esula dagli scopi di questo tutorial, ma che rappresenta un'applicazione interessante per il linguaggio naturale nell'industria.

In un'altra direzione, se si sa a priori che un documento riguarda un particolare dominio o un insieme di argomenti, è possibile vincolare i significati restituiti da WordNet. Nell'esempio che segue, vogliamo considerare i risultati della NLU che si riferiscono al settore finanziario e bancario:

In [36]:
domains = ["finance", "banking"]
sentence = nlp("I want to withdraw 5,000 euros.")

enriched_sent = []

for token in sentence:
    # get synsets within the desired domains
    synsets = token._.wordnet.wordnet_synsets_for_domain(domains)

    if synsets:
        lemmas_for_synset = []

        for s in synsets:
            # get synset variants and add to the enriched sentence
            lemmas_for_synset.extend(s.lemma_names())
            enriched_sent.append("({})".format("|".join(set(lemmas_for_synset))))
    else:
        enriched_sent.append(token.text)

print(" ".join(enriched_sent))

AttributeError: [E046] Can't retrieve unregistered extension attribute 'wordnet'. Did you forget to call the `set_extension` method?

L'esempio può sembrare semplice ma, se si gioca con l'elenco dei domini, si scopre che i risultati hanno una sorta di esplosione combinatoria quando vengono eseguiti senza vincoli ragionevoli. Immaginate di avere un grafo della conoscenza con milioni di elementi: vorreste limitare le ricerche dove possibile per evitare che ogni query richieda giorni/settimane/mesi/anni di calcolo.

A volte i problemi che si incontrano quando si cerca di capire un testo - o meglio ancora quando si cerca di capire un corpus (un insieme di dati con molti testi correlati) - diventano così complessi che è necessario prima visualizzarli. Ecco una visualizzazione interattiva per la comprensione dei testi: scattertext, frutto del genio di Jason Kessler. Da installare:

```conda install -c conda-forge scattertext```
Analizziamo i dati di testo delle convention di partito durante le elezioni presidenziali americane del 2012. L'esecuzione potrebbe richiedere uno o due minuti, ma i risultati di tutta questa elaborazione di numeri valgono l'attesa.

In [35]:
!pip install scattertext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=9df6cbb61176366ef1d318503525d02da08f26744051b296f79551297a9523f1
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext


In [39]:
import scattertext as st

if "merge_entities" not in nlp.pipe_names:
    nlp.add_pipe(nlp.create_pipe("merge_entities"))

if "merge_noun_chunks" not in nlp.pipe_names:
    nlp.add_pipe(nlp.create_pipe("merge_noun_chunks"))

convention_df = st.SampleCorpora.ConventionData2012.get_data()
corpus = st.CorpusFromPandas(convention_df,
                             category_col="party",
                             text_col="text",
                             nlp=nlp).build()

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <function merge_entities at 0x7e38cbb3fc70> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.

Una volta pronto il `corpus', generare una visualizzazione interattiva in HTML:



In [40]:
html = st.produce_scattertext_explorer(
    corpus,
    category="democrat",
    category_name="Democratic",
    not_category_name="Republican",
    width_in_pixels=1000,
    metadata=convention_df["speaker"]
)

NameError: name 'corpus' is not defined

Ora renderizzeremo l'HTML: dategli un minuto o due per caricarlo, ne vale la pena...

In [41]:
from IPython.display import IFrame
from IPython.core.display import display, HTML
import sys

IN_COLAB = "google.colab" in sys.modules
print(IN_COLAB)

True


In [42]:
if IN_COLAB:
    display(HTML("<style>.container { width:98% !important; }</style>"))
    display(HTML(html))

NameError: name 'html' is not defined

In [ ]:
file_name = "foo.html"

with open(file_name, "wb") as f:
    f.write(html.encode("utf-8"))

IFrame(src=file_name, width = 1200, height=700)